# Introduction
Welcome to the introduction to the tutorial. In this part, we will prepare ourself for running molecular dynamics simulation using the open-source software [LAMMPS](https://www.lammps.org/#gsc.tab=0).

The topics are listed below:
1. What is molecular dynamics simulation?
2. What is radiation damage and why we need molecular dynamics simulation?
3. What special care is needed for radiation simulation?
4. What softwares and packages do we need for this tutorial?

## 1. What is molecular dynamics simulation?
In simple words, molecular dynamics simulation is tracking the motion of every atoms inside a system based on Newton's equation of motion
$$F=ma$$

A few things to note:
- The smallest unit is an atom so there is no quantum effect from electrons. Sometimes this is referred as atomistic simulation.
- The force between atoms are designated by **Force Field** or **Interatomic Potential**. Usually, there are two broad categories: non-reactive and reactive force field. Non-reactive means that the connectivity of bonds and molecules are preserved. This is usually enough to simulate physical phenomenon like solution. In contrast, reactive force field means that bonds are allowed to break and reform during the simulation, which opens up the possibility of simulating chemical reactions. Force field itself can be quite complex and the functional form highly depends on the material system you want to study (metals, ceramics, polymers, biomaterials, etc).
- Knowing the acceleration from the force, numerical techniques such as Velocity Verlet are used to update velocity and position for every timestep.
- Typically, the flowchart of a molecular dynamics simulation is
1. Prepare structure
2. Choose force field
3. Energy minimzation
4. Equilibration (or non-equilibrium simulation)
5. Post-analysis

For writing a basic molecular dynamics simulation code using python, please read https://doi.org/10.1119/10.0000654.

## 2. What is radiation damage and why we need molecular dynamics simulation?
Radiation damage is the study of structural and property changes caused by high energy radiation (light or particles). This is particularly important for nuclear engineering and space applications. When an energetic particle penetrates to a material, it typically interacts with both the nucleus and electrons. The incoming particle interacts mostly with the electrons and then as it slows down, it interacts more with the nucleus. The later stage is typically referred as collision cascade where many-body interactions take place to create structural changes. If we are interested in bulk materials (not surface reactions), PKA model is used. PKA refers to primary (i.e. the first) knock-on atom which is displaced due to nuclear collision with the incoming particle. After the collision, the incoming particle is assumed to be scattered off the region. The localized structural change is assumed to be the direct consequence of PKA-induced collision cascade.

The process for radiation damage is a multiscale problem. Sometimes, the property changes take months or years to be detected. However, for the primary radiation damage, where time effect is less important (i.e. thermal diffusion is ignored), it typically lasts for picoseconds. In addition, for metals, electrons are free to move so most structural changes are nuclear (or atomic) displacements. This is why molecular dynamics simulation is used because it captures the many-body interactions inherently at femtoseconds and electrons can be safely ignored. For covalent and ionic materials, electronic effect plays a more important role in chemical and structural changes. Generally, if we only focus on the low-energy regime where nuclear collisions dominate, PKA model should be valid. To determine the threshold, software such as SRIM can be used to calculate the nuclear and electronic stopping power, which measures how much energy is dissipated per unit length of travel. You can search SRIM or stopping power for more information.

## 3. What special care is needed for radiation simulation?
Different from other types of molecular dynamics simulation, radiation means that
1. Only reactive force field can be used because of the high-energy nature (bonds breaking and formation).
2. Atoms will approach closely to each other or even overlap due to high kinetic energy. This short-range interaction is not considered for most force field and a good protocal is to spline with [ZBL potential](https://docs.lammps.org/pair_zbl.html).
3. The intial high kinetic energy means we need a very small timestep and as the system cools down, a large timestep can be used. The best way is to invoke a variable timestep scheme to save computational time and maintain numerical stability.

Since we are interested in bulk changes, periodic boundary condition is invoked for all three directions. We only use NVE ensemble since this is an non-equilibrium process. However, in practice, we expect heat dissipation from the system but we can only simulate a finite size. To address this issue, a heat dissipation layer is added at the outer boundary. Lastly, we need a relatively large system size to avoid an recoil atoms entering the heat dissipation layer and prevent unphysical temperature rise (e.g., the temperature should not be close to melting temperature). A simple size scaling test can be used.

For more detailed discussion, please read http://hdl.handle.net/10138/337071.

## 4. What softwares and packages do we need for this tutorial?

### Softwares
1. Download [Visual Studio Code](https://code.visualstudio.com/). For Windows users, you can click https://code.visualstudio.com/Download.
2. Please make sure you have installed Python in your laptop. For Windows users, you can click https://www.python.org/ftp/python/3.12.2/python-3.12.2-amd64.exe. Make sure to click "Add python.exe to PATH" during installation.
3. All simulations are run using [LAMMPS](https://www.lammps.org/#gsc.tab=0). For Windows users, please go to https://rpm.lammps.org/windows/. In particular, we will be use "2024-02-08 03:19	  LAMMPS-64bit-latest.exe". Or you can click https://rpm.lammps.org/windows/LAMMPS-64bit-latest.exe to download directly. Force field used is contained in LAMMPS folder after you install it. Search "LAMMPS" and pin it for quick access.
4. For visulization and post-analysis, we will use [OVITO](https://www.ovito.org/). You can click https://www.ovito.org/#download to download directly.

### Packages
1. For Visual Studio Code, please find and install Extenstions:
- Python
- Lammps Syntax Highlighting

2. For Python, please install Packages:
- Matplotlib
- NumPy
- pandas
- ovito (Python module of OVITO software)

Open **Terminal** and type ``python -m pip install ipykernel --upgrade``.
This makes sure to run Python in cells.

The python package installation can be easily installed using PyPI. Just run the following cells.

In [1]:
# Install Python Packages
!python -m pip install -U pip
!python -m pip install -U matplotlib
!python -m pip install -U pandas
!python -m pip install -U numpy
!python -m pip install -U ovito

   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.8 MB ? eta -:--:--
   ---- ----------------------------------- 0.8/7.8 MB 1.6 MB/s eta 0:00:05
   ----- ---------------------------------- 1.0/7.8 MB 1.5 MB/s eta 0:00:05
   --------- ------------------------------ 1.8/7.8 MB 2.5 MB/s eta 0:00:03
   -------------------- ------------------- 3.9/7.8 MB 4.0 MB/s eta 0:00:01
   -------------------------------------- - 7.6/7.8 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 7.8/7.8 MB 6.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 30.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------- ----------------- 7.1/12.6 MB 33.6 MB/s eta 0:00:01
   --------------------------------------- 